In [1]:
import pandas as pd

In [2]:
import requests
import warnings
warnings.filterwarnings("ignore")

In [3]:
#pip install fake-useragent

In [4]:
#pip install --upgrade fake-useragent

In [5]:
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'

In [6]:
from selenium import webdriver #робот, который проходит по страничкам в хроме
from selenium.webdriver.common.keys import Keys #ввод данных в поле "Поиск"
from selenium import webdriver 
from bs4 import BeautifulSoup
import random
from time import sleep

In [7]:
#pip install selenium

In [8]:
#pip install webdriver_manager

In [9]:
#pip install -U selenium

In [10]:
driver = webdriver.Chrome()

# Вытаскиваем все ссылки с первой страницы

In [11]:
url = 'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'

In [12]:
def get_links(first_link):
    
    driver.get(first_link)
    
    my_page = BeautifulSoup(driver.page_source)
    
    urls = []
    
    for link in my_page.find_all('a'):
        try:
            if '/sale/flat/' in link.get('href') and '/export/pdf' not in link.get('href'):
                urls.append(link.get('href'))
        except Exception:
            pass
    return urls

In [13]:
all_links = get_links(url)
print(len(all_links))

84


In [68]:
def get_links_by_hand():
    
    my_page = BeautifulSoup(driver.page_source)
    
    urls = []
    
    for link in my_page.find_all('a'):
        try:
            if '/sale/flat/' in link.get('href') and '/export/pdf' not in link.get('href'):
                urls.append(link.get('href'))
        except Exception:
            pass
    return urls

In [81]:
all_links = all_links + get_links_by_hand()
print(len(all_links))

1176


# Начинаем собирать данные о квартирах

## Данные о доме

## Вытаскиваем гео данные

In [14]:
def get_longitude(page_data):
    long_data = []
    
    for i in page_data.find_all('script'):
        try:
            if 'll=' in i.get('src'):
                long_data.append(str(i).split('&amp')[1].replace(';','')[3:].split(',')[0])
                long_data.append(str(i).split('&amp')[1].replace(';','')[3:].split(',')[1])
        except Exception:
            pass
    return list(set(long_data))

In [15]:
def flat_data(flat_link):
    driver.get(flat_link)
    sleep(10)
    
    my_page = BeautifulSoup(driver.page_source)
    
    house_data = my_page.find_all('span', {'class' : 'a10a3f92e9--color_black_100--Ephi7 a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_bold--BbhnX a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY'})
    
    house_char = []
    
    for i in range(len(house_data)):
        house_char.append(house_data[i].text.split()[0])

    total_price = []
    
    price = my_page.find_all('span', {'class' : 'a10a3f92e9--color_black_100--Ephi7 a10a3f92e9--lineHeight_9u--limEs a10a3f92e9--fontWeight_bold--BbhnX a10a3f92e9--fontSize_28px--P1gR4 a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY'})

    for i in price:
        if '₽' in str(i):
            total_price.append(i)
            
    house_char = house_char[:5]
            
    house_char.append(str(total_price[0]).split('>')[1].split('<')[0].replace("\xa0",''))
    
    house_char.extend(get_longitude(my_page))
    
    house_char.append(flat_link)
    
    
    return house_char

In [16]:
smth = flat_data(all_links[1])
smth

['49,3',
 '29,4',
 '20',
 '2023',
 'Не',
 '50488727₽',
 '55.73448837',
 '37.52466149',
 'https://www.cian.ru/sale/flat/283856120/']

Микрорайоны, расстояние до метро, будем брать всю Москву

Выгрузить 20 к 

Попробовать написать сайт на стримлите

In [17]:
driver.close()

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=116.0.5845.187)
Stacktrace:
0   chromedriver                        0x000000010287265c chromedriver + 4318812
1   chromedriver                        0x000000010286ad00 chromedriver + 4287744
2   chromedriver                        0x000000010249c7ec chromedriver + 296940
3   chromedriver                        0x0000000102481a48 chromedriver + 186952
4   chromedriver                        0x0000000102482c30 chromedriver + 191536
5   chromedriver                        0x0000000102472dec chromedriver + 126444
6   chromedriver                        0x0000000102473afc chromedriver + 129788
7   chromedriver                        0x0000000102505890 chromedriver + 727184
8   chromedriver                        0x00000001024ce178 chromedriver + 500088
9   chromedriver                        0x00000001024cefc0 chromed